In [59]:
import pandas as pd
from sodapy import Socrata
from dotenv import load_dotenv
import os
import psycopg2 as pg2

In [60]:
load_dotenv()
myapptoken = os.getenv('MYAPPTOKEN')
postgres_user = os.getenv('POSTGRES_USER')
postgres_password = os.getenv('POSTGRES_PASSWORD')
client = Socrata("data.ny.gov",
                 myapptoken
                 )


In [61]:
results = client.get("qzve-kjga", where="plaza_id = 30", limit=100000)
results_df = pd.DataFrame.from_records(results)

In [62]:
results_df

,plaza_id,date,hour,direction,vehicles_e_zpass,vehicles_vtoll
0,30,2024-10-19T00:00:00.000,0,I,1604,285
1,30,2024-10-19T00:00:00.000,0,O,2903,435
2,30,2024-10-19T00:00:00.000,1,I,1140,257
3,30,2024-10-19T00:00:00.000,1,O,1899,335
4,30,2024-10-19T00:00:00.000,2,I,760,151
...,...,...,...,...,...,...
93534,30,2017-09-03T00:00:00.000,19,O,4176,529
93535,30,2017-09-03T00:00:00.000,20,O,4133,547
93536,30,2017-09-03T00:00:00.000,21,O,3703,506
93537,30,2017-09-03T00:00:00.000,22,O,3364,498


In [63]:
connection = pg2.connect(dbname='MTA Open Data', user=postgres_user, password=postgres_password)

In [64]:
cur = connection.cursor()

create_table_query = """ 
CREATE TABLE IF NOT EXISTS traffic 
    (   
        id SERIAL PRIMARY KEY,
        plaza_id SMALLINT,
        date TIMESTAMP,
        hour SMALLINT,
        direction text,
        vehicles_ezpass integer,
        vehicles_vtoll integer
    )

"""

cur.execute(create_table_query)
connection.commit()
cur.close()

In [65]:
insert_cur = connection.cursor()

insert_query = """
    INSERT INTO traffic (plaza_id, date, hour, direction, vehicles_ezpass, vehicles_vtoll)
    VALUES
    (%s,%s,%s,%s,%s,%s)

"""
for rows in results_df.itertuples():
    insert_cur.execute(insert_query, (rows.plaza_id, rows.date, rows.hour, rows.direction, rows.vehicles_e_zpass, rows.vehicles_vtoll))
connection.commit()
insert_cur.close()

In [66]:
change_direction_cur = connection.cursor()

update_query = """
    UPDATE traffic 
    SET direction = CASE
        WHEN direction = 'I' THEN 'Inbound'
        WHEN direction = 'O' THEN 'Outbound'
        ELSE direction
    END
"""

change_direction_cur.execute(change_direction_query)
connection.commit()
change_direction_cur.close()

In [67]:
total_vehicle_cursor = connection.cursor()

total_vehicle_query = """
    ALTER TABLE traffic 
    ADD COLUMN total_vehicle integer
"""
total_vehicle_cursor.execute(total_vehicle_query)

update_query = " UPDATE traffic SET total_vehicle = vehicles_ezpass + vehicles_vtoll"

total_vehicle_cursor.execute(update_query)
connection.commit()
total_vehicle_cursor.close()

In [68]:
connection.close()